In [1]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
# method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_EXACT
method = METHOD_SCALAR_QUANTIZE

In [3]:
feedback_bits = 256
ncodebooks=64
ncentroids=256
train_sam_num = 3000 # 训练集样本数

In [4]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= 'data_to_fc_e39_7999.npy'
    featurepath_train= 'feature_e39_7999.npy'
    data_to_fcpath_test = 'data_to_fc_e39_7999.npy'
    featurepath_test = 'feature_e39_7999.npy'
elif host_name == 'jm-System-Product-Name':
    dir_train = '/data/hdr/transformer_data/joined'
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'data_to_fc_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'feature_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'data_to_fc_test.npy'
    featurepath_test = 'feature_test.npy'
    y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dir_train")


weightpath = 'encoder_fcw.npy'
biaspath = 'encoder_fcb.npy'



In [5]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])


running method:  ScalarQuantize


In [6]:
x_test = np.load(dir_train+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [7]:
print(y_out_last)
print(y_out_last.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
np.save(dir_result+'/'+method+'fc1_fb%i_cb%i_ct%i.npy' % (feedback_bits, ncodebooks, ncentroids), y_out_last)
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[-5.3043962e-01  3.0113718e-01 -5.6065595e-01 ... -8.4870778e-02
   1.4455784e-03  8.5218197e-01]
 [-6.6278100e-01  1.4982341e-01 -3.2295024e-01 ... -3.9204141e-01
  -6.4814761e-02  7.6440579e-01]
 [-2.2893736e-01  3.2937548e-01 -2.5860965e-01 ... -5.2163786e-01
   6.5915391e-02  1.3257462e+00]
 ...
 [-1.8937704e-01  4.5295641e-01 -5.5048931e-01 ... -1.9490215e-01
  -3.1064910e-01  1.0301033e+00]
 [-2.4727842e-01  3.0374262e-01 -2.6777589e-01 ...  3.8214482e-02
  -9.7049430e-02  9.7844082e-01]
 [-1.0404379e+00  5.7632565e-01 -4.7084343e-01 ... -3.5279366e-01
  -2.9770201e-01  1.6130158e+00]]
(32000, 256)
